# Funcion 1


In [4]:
import pandas as pd


def PlayTimeGenre(genero: str):
    try:
        # Leemos el archivo parquet
        df_genero = pd.read_parquet("Src/endpoint1.parquet")
        
        # Filtrar el DataFrame por el género especificado
        df_genero = df_genero[df_genero["genres"] == genero]
        
        # Encontrar el año con más horas jugadas para el género
        año_con_mas_horas = list(df_genero[df_genero["playtime"] == df_genero["playtime"].max()]["release"])[0]
        
        return {f"Año de lanzamiento con más horas jugadas para {genero}": año_con_mas_horas}
    
    except Exception as e:
        return {"error": str(e)}


In [9]:
PlayTimeGenre('Puzzle')

{'Año de lanzamiento con más horas jugadas para Puzzle': 2009}

# Funcion 2

In [3]:
def userforgenre(genero: str):
    #Leemos el parquet
    endpoint2 = pd.read_parquet('src/endpoint2.parquet')
    
    #Convertimos los minutos a horas
    endpoint2['playtime'] = round(endpoint2['playtime']/60,2)
    
    #Filtramos el género solicitado
    endpoint2_genero = endpoint2[endpoint2['genres'] == genero]
    
    #Buscamos el usuario con más horas en el género
    usuario_con_mas_horas = endpoint2_genero.loc[endpoint2_genero['playtime'].idxmax()]['user_id']
    
    #Agrupamos por año 
    horas_por_año_usuario = endpoint2_genero[endpoint2_genero['user_id'] == usuario_con_mas_horas]
    horas_por_año_usuario = horas_por_año_usuario.groupby('release')['playtime'].sum().reset_index()
    horas_por_año_usuario = horas_por_año_usuario.rename(columns = {'release': 'Año','playtime':'Horas'})

    #Creamos la listita de acumulacion de horas jugadas por año
    lista_horas_por_año = horas_por_año_usuario.to_dict(orient='records')
    return {f"Usuario con más horas jugadas para {genero}": usuario_con_mas_horas,
        "Horas jugadas": lista_horas_por_año}

In [4]:
userforgenre('Action')

{'Usuario con más horas jugadas para Action': 'Evilutional',
 'Horas jugadas': [{'Año': 2009, 'Horas': 33.95},
  {'Año': 2010, 'Horas': 68.37},
  {'Año': 2011, 'Horas': 32.8},
  {'Año': 2012, 'Horas': 11349.85},
  {'Año': 2013, 'Horas': 1162.1},
  {'Año': 2014, 'Horas': 403.77},
  {'Año': 2015, 'Horas': 1.87},
  {'Año': 2016, 'Horas': 21.52},
  {'Año': 2017, 'Horas': 181.57}]}

# Funcion 3


In [10]:
def UsersRecommend(year: int):
    try:
        # Leemos el archivo
        df = pd.read_parquet('src/reviews.parquet')
        #print("DataFrame cargado:", df.head())  # Verificamos los primeros registros del DataFrame
        
        # Filtramos por año
        df_year = df[df['posted_year'] == year]
        #print("Registros para el año", year, ":", len(df_year))  # Verificamos cuántos registros hay para el año específico
        
        # Filtramos por recomendaciones positivas/neutrales
        df_recommend = df_year[df_year['recommend'] == True]
        df_sentiment = df_recommend[df_recommend['sentiment_analysis'].isin([2, 1])]
        #print("Registros con recomendaciones positivas/neutrales:", len(df_sentiment))  # Verificamos cuántos registros cumplen con los criterios de recomendación y sentimiento
        
        # Excluimos títulos 'Otros'
        df_filtered = df_sentiment[df_sentiment['title'] != 'Otros']
        # print("Registros después de excluir 'Otros':", len(df_filtered))  # Verificamos cuántos registros quedan después de excluir 'Otros'
        
        # Agrupamos por título y contamos las recomendaciones
        recommendations = df_filtered.groupby('title')['recommend'].sum()
        #print("Recomendaciones por juego:", recommendations)  # Verificamos el conteo de recomendaciones por juego
        
        # Ordenamos las recomendaciones por número de recomendaciones 
        recommendations_sorted = recommendations.sort_values(ascending=False)
        
        # Tomamos los tres primeros juegos
        top_3 = recommendations_sorted.head(3)
        #print("Top 3 juegos recomendados:", top_3)  # Verificamos el top 3 de juegos recomendados
        
        # Verificamos si hay suficientes juegos recomendados
        if len(top_3) >= 3:
            # Creamos una lista de diccionarios para los tres primeros juegos
            result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(top_3.index)]
        else:
            # Si no hay suficientes juegos, devolvemos un mensaje de datos insuficientes
            result = 'Datos insuficientes'
        
        return result
    
    except Exception as e:
        # Capturamos cualquier excepción y la devolvemos como un diccionario
        return {"error": str(e)}
    

In [11]:
UsersRecommend(2017)

'Datos insuficientes'

# Funcion 4


In [12]:
def UsersWorstDeveloper(year:int):
    try:
        # Leemos el archivo
        df = pd.read_parquet('src/reviews.parquet')
        #print("DataFrame cargado:", df.head())  # Verificamos los primeros registros del DataFrame
        
        # Filtramos por año
        df_year = df[df['posted_year'] == year]
        #print("Registros para el año", year, ":", len(df_year))  # Verificamos cuántos registros hay para el año específico
        
        # Filtramos por recomendaciones positivas/neutrales
        df_recommend = df_year[df_year['recommend'] == False]
        df_sentiment = df_recommend[df_recommend['sentiment_analysis'] == 0]
        #print("Registros con recomendaciones positivas/neutrales:", len(df_sentiment))  # Verificamos cuántos registros cumplen con los criterios de recomendación y sentimiento
        
        # Excluimos títulos 'Otros'
        df_filtered = df_sentiment[df_sentiment['developer'] != 'Otros']
        # print("Registros después de excluir 'Otros':", len(df_filtered))  # Verificamos cuántos registros quedan después de excluir 'Otros'
        
        # Agrupamos por título y contamos las recomendaciones
        negative_recommendations = df_filtered.groupby('developer')['recommend'].sum()
        #print("Recomendaciones por juego:", recommendations)  # Verificamos el conteo de recomendaciones por juego
        
        # Ordenamos las recomendaciones por número de recomendaciones 
        negative_recommendations_sorted = negative_recommendations.sort_values(ascending=False)
        
        # Tomamos los tres primeros juegos
        top_3 = negative_recommendations_sorted.head(3)
        #print("Top 3 juegos recomendados:", top_3)  # Verificamos el top 3 de juegos recomendados
        
        # Verificamos si hay suficientes juegos recomendados
        if len(top_3) >= 3:
            # Creamos una lista de diccionarios para los tres primeros juegos
            result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(top_3.index)]
        else:
            # Si no hay suficientes juegos, devolvemos un mensaje de datos insuficientes
            result = 'Datos insuficientes'
        
        return result
    
    except Exception as e:
        # Capturamos cualquier excepción y la devolvemos como un diccionario
        return {"error": str(e)}
    

In [14]:
UsersWorstDeveloper(2015)

[{'Puesto 1': '10th Art Studio,Adventure Productions'},
 {'Puesto 2': 'Red Duck Inc.'},
 {'Puesto 3': 'Reality Pump'}]

# Funcion 5


In [15]:
def SentimentAnalysis(desarrolladora:str):
    try:
        #Leemos el archivo    
        df_sentiment = pd.read_parquet('src/reviews.parquet')
        
        #Filtramos el df con la desarrolladora
        df_developer = df_sentiment[df_sentiment['developer'] == desarrolladora]
        
        #Contabilizamos las reviews
        positive_count = (df_developer['sentiment_analysis'] == 2).sum()
        neutral_count = (df_developer['sentiment_analysis'] == 1).sum()
        negative_count = (df_developer['sentiment_analysis'] == 0).sum()
        
        #Creamos el diccionario solicitado:
        result_dicc = {
            'Desarrolladora': desarrolladora,
            'Reviews Positivas': positive_count,
            'Reviews Neutras': neutral_count,
            'Reviews Negativas': negative_count
        }
        return result_dicc
        
    except Exception as e:
        # Capturamos cualquier excepción y la devolvemos como un diccionario
        return {"error": str(e)}

In [16]:
SentimentAnalysis('Ubisoft')

{'Desarrolladora': 'Ubisoft',
 'Reviews Positivas': 83,
 'Reviews Neutras': 1,
 'Reviews Negativas': 35}